In [1]:
import os
import re
import numpy
import pandas as pd
# import PyPDF2
import requests
from bs4 import BeautifulSoup
from googlesearch import search

In [2]:

file = "Quantum_Machine_Learning_for_6G_Communication_Networks:_State-of-the-Art_and_Vision_for_the_Future" 
path = os.getcwd() + "/references/" + file +".txt"
csv_path = os.getcwd() + "/data/" + file +".csv"

In [8]:
class ResearchPaperList:
    def __init__(self):
        self.paper_data = ["title","author","date",0,"DOI",0,"publisher","url"]     
        self.ref_path = os.getcwd() + "/references/.txt"
        self.csv_path = os.getcwd() + "/data/Research_Paper_Data.csv"
        self.id_path = os.getcwd() + "/data/Research_Paper_List.csv"
        self.link_path = ""
        self.ref_data = ""
        self.link_data = ""
        self.open_id()
        self.open_csv()
    
    def open_id(self):
        if(not os.path.exists(self.id_path)):
            print("csv file not exists")
            return
        self.id_data = pd.read_csv(self.id_path)
        return
    
    def open_csv(self):
        if(not os.path.exists(self.csv_path)):
            print("csv file not exists")
            return False
        self.csv_data = pd.read_csv(self.csv_path, index_col = 0)
        return True
    
    def open_link(self):
        if(not os.path.exists(self.link_path)):
            print("csv file not exists")
            return False
        self.link_data = pd.read_csv(self.link_path, index_col = 0)
        return True
    
    def save_id_data(self):
        self.id_data.to_csv(self.id_path, sep=',', na_rep='NaN', index = False)
        
    def save_csv_data(self):
        self.csv_data.to_csv(self.csv_path,sep=',', na_rep='NaN')
    
    def save_link_data(self):
        self.link_data.to_csv(self.link_path,sep=',', na_rep='NaN')

    def get_id_by_DOI(self,doi):
        result = self.id_data.loc[q1.id_data["DOI"] == doi].values.tolist()
        if(not len(result)):
            return -1
        return result[0][0]

    def create_id(self, doi, title):
        result = [len(self.id_data), doi, title]
        self.id_data.loc[len(self.id_data)] = result
#         self.save_id_data()
        return result[0]

    
    
    def open_ref(self):
        if(not os.path.exists(self.ref_path)):
            print("file not exists")
            return False
        if(self.ref_data):
            self.ref_data.close()
        self.ref_data =  open(self.ref_path,"r", encoding='utf-8') 
        return True
        
    def print_ref(self, head_count = 5000):
        if not self.ref_data:
            self.open_ref()
        if(self.ref_data):
            while True:
                if(head_count == 0) : break
                line = self.ref_data.readline()
                if not line: break
                print(line) 
                head_count -= 1
    
    def close_ref(self):
        if(self.ref_data):
            self.ref_data.close()
            
    def set_data(self, paper_data, reference_list):
        self.paper_data = paper_data
        self.ref_path = os.getcwd() + "/references/" + reference_list +".txt"
        self.open_ref()
        paper_id = self.get_id_by_DOI(paper_data[4])
        if(paper_id == -1):
            paper_id = self.create_id(paper_data[4], paper_data[0])
            print(paper_id)
            self.save_id_data()
        self.link_path = os.getcwd() + "/data/"+ str(paper_id) +".csv"
        if (not self.open_link()):
            self.link_data = pd.DataFrame(columns=["id","title"])
            self.link_data.loc[0] = [paper_id] + [paper_data[0]]
            self.csv_data.loc[paper_id] = paper_data + [""]
            self.save_csv_data()
            self.save_link_data()
        
                
    def check_index(self, line,index):
        if(line[0]=="[" and line[1:1+len(str(index))] == str(index)):
            return True
        else:
            return False

    def get_paper_html(self, search_url):
        response = requests.get(search_url)
        if response.status_code == 200:
            html = response.text
            soup = BeautifulSoup(html, 'html.parser')
            return soup
        else:
            return -1

    def get_citations(self, html):
        html = str(html)
        data = 0
        temp = html.split('"citationCount":"')
        if(len(temp)>1):
            data = int(temp[1].split('"')[0])
        return data

    def get_INSPEC(self, html):
        html = str(html)
        data = 0
        temp = html.split('"accessionNumber":"')
        if(len(temp)>1):
            data = int(temp[1].split('"')[0])
        return data

    def get_DOI(self, html):
        html = str(html)
        data = ""
        temp = html.split('"doi":"')
        if(len(temp)>1):
            data = temp[1].split('"')[0]
        return data

    def get_date(self, html):
        html = str(html)
        data = ""
        temp = html.split('"dateOfInsertion":"')
        if(len(temp)>1):
            data = temp[1].split('"')[0]
        return data
        
    def handle_data(self, csv_data, current,index):
        temp = ["","","",0,"",0,"","",""]
        temp_link = [-1,""]
        current =current.replace("\n","")
#         print("index ",index,"\n",current)
#         parse = re.split('“|”',current)
        parse = re.split('‘‘|’’',current)
        if(len(parse)<2):
            return
        temp[0] = parse[1][:-1]
        temp_link[1] = temp[0]
        temp[1] = parse[0]
        is_url = False
        if("IEEE" in parse[2]):
            temp[6] = "IEEE"
            for search_url in search(temp[0], tld="co.in", num=5, stop=5, pause=1):
                if('ieee' in search_url):
                    temp[7] = search_url
                    is_url = True
                    break;
        if(is_url):
            html = self.get_paper_html(temp[7])
            temp[4] = self.get_DOI(html)
            temp_link[0] = self.get_id_by_DOI(temp[4])
            if(temp_link[0] == -1):
                temp_link[0] = self.create_id(temp[4], temp[0])
                temp[5] = self.get_citations(html)
                temp[3] = self.get_INSPEC(html)
                temp[2] = self.get_date(html)
                self.csv_data.loc[temp_link[0]] = temp
                
#                 self.save_csv_data()
#                 self.save_id_data()
#                 self.save_link_data()
        print("idx ", index, " ", temp_link, " ", temp)
        self.link_data.loc[index] = temp_link
        

    def extract_reference(self):
        index = 1
        current = ""
        
        while True:
            
            line = self.ref_data.readline()
#             print(line)
            if not line: break
            if(self.check_index(line, index+1)):
                self.handle_data(self.csv_data,current[3+len(str(index)):],index)
                current = line
                index += 1
            else:
                current += line
        self.ref_data.close()
        self.save_id_data()
        self.save_csv_data()
        self.save_link_data()
    
    
                
            

In [9]:
q1 = ResearchPaperList()
q1.set_data(["Quantum machine learning for 6G communica-tion networks: State-of-the-art and vision for the future",
             "S. J. Nawaz, S. K. Sharma, S. Wyne, M. N. Patwary, andM. Asaduzzaman, ",
             "15 April 2019",
             18609246,
             "10.1109/ACCESS.2019.2909490",
             170,
             "IEEE",
             "https://ieeexplore.ieee.org/document/8681450"],
            "ref2")


In [10]:
print(len(q1.id_data))
# d1 = q1.id_data.loc[q1.id_data["DOI"] == "10.1109/OJCOMS.2020.3010270"].values.tolist()
# print(d1)
print(q1.get_id_by_DOI("10.1109/OJCOMS.20202.3010270"))


51
-1


In [11]:
q1.extract_reference()

idx  2   [51, 'A survey of 5G network: Architecture andemerging technologies']   ['A survey of 5G network: Architecture andemerging technologies', 'A. Gupta and E. R. K. Jha, ', '', 0, '', 0, 'IEEE', 'https://ieeexplore.ieee.org/document/7169508', '']
idx  3   [52, 'Towards massive connectivity support for scal-able mMTC communications in 5G networks']   ['Towards massive connectivity support for scal-able mMTC communications in 5G networks', 'C. Bockelmann et al., ', '18 June 2018', 17829625, '10.1109/ACCESS.2018.2837382', 131, 'IEEE', 'http://ieeexplore.ieee.org/document/8360103/', '']
idx  4   [-1, 'Towards massive machinetype communications in ultra-dense cellular IoT networks: Currentissues and machine learning-assisted solutions']   ['Towards massive machinetype communications in ultra-dense cellular IoT networks: Currentissues and machine learning-assisted solutions', 'S. K. Sharma and X. Wang. (Aug. 2018). ', '', 0, '', 0, '', '', '']
idx  5   [-1, 'Open issues and beyond 5G'] 

idx  32   [65, 'EXIT-chart aided quantum code design improvesthe normalised throughput of realistic quantum devices']   ['EXIT-chart aided quantum code design improvesthe normalised throughput of realistic quantum devices', 'H. V. Nguyen et al., ', '06 May 2019', 16606904, '10.1109/ACCESS.2016.2591910', 20, 'IEEE', 'https://ieeexplore.ieee.org/document/7515170', '']
idx  33   [66, 'Quantum-aided multi-objective routing optimizationusing back-tracing-aided dynamic programming']   ['Quantum-aided multi-objective routing optimizationusing back-tracing-aided dynamic programming', 'D. Alanis et al., ', '06 May 2019', 17991417, '10.1109/TVT.2018.2822626', 4, 'IEEE', 'http://ieeexplore.ieee.org/document/8329527/', '']
idx  34   [67, 'Quantum turbo decoding for quantum channelsexhibiting memory']   ['Quantum turbo decoding for quantum channelsexhibiting memory', 'M. A. M. Izhar et al., ', '06 May 2019', 17649172, '10.1109/ACCESS.2018.2808373', 3, 'IEEE', 'https://ieeexplore.ieee.org/abstract/d

idx  61   [84, 'Analysis of spectrum occupancy usingmachine learning algorithms']   ['Analysis of spectrum occupancy usingmachine learning algorithms', 'F. Azmat, Y. Chen, and N. Stocks, ', '15 September 2016', 16316592, '10.1109/TVT.2015.2487047', 64, 'IEEE', 'https://ieeexplore.ieee.org/document/7289481', '']
idx  62   [85, 'A survey of energy-efficient techniques for 5G networks and challengesahead']   ['A survey of energy-efficient techniques for 5G networks and challengesahead', 'S. Buzzi, C.-L. I, T. E. Klein, H. V. Poor, C. Yang, and A. Zappone,', '10 May 2016', 15985567, '10.1109/JSAC.2016.2550338', 463, 'IEEE', 'https://ieeexplore.ieee.org/document/7446253', '']
idx  63   [86, 'A new wireless communication paradigm through software-controlled metasurfaces']   ['A new wireless communication paradigm through software-controlled metasurfaces', 'C. Liaskos, S. Nie, A. Tsioliaridou, A. Pitsillides, S. Ioannidis, andI. Akyildiz, ', '17 September 2018', 18128617, '10.1109/MCOM.2018.1

idx  89   [51, 'A com-prehensive survey of recent advancements in molecular communication']   ['A com-prehensive survey of recent advancements in molecular communication', 'N. Farsad, H. B. Yilmaz, A. Eckford, C. B. Chae, and W. Guo, ', '', 0, '', 0, 'IEEE', 'https://ieeexplore.ieee.org/document/7405285', '']
idx  90   [-1, 'Receiver design in molecular communica-tions: An approach based on artificial neural networks']   ['Receiver design in molecular communica-tions: An approach based on artificial neural networks', 'X. Qian and M. Di Renzo, ', '', 0, '', 0, '', '', '']
idx  91   [-1, 'WLAN–based indoor localization using neuralnetworks']   ['WLAN–based indoor localization using neuralnetworks', 'F. Saleem and S. Wyne, ', '', 0, '', 0, '', '', '']
idx  92   [-1, 'A deep learning framework for optimization of MISO downlinkbeamforming']   ['A deep learning framework for optimization of MISO downlinkbeamforming', 'W. Xia, G. Zheng, Y. Zhu, J. Zhang, J. Wang, and A. P. Petropulu.(2019). '

idx  121   [116, 'A survey on quantum channelcapacities']   ['A survey on quantum channelcapacities', 'L. Gyongyosi, S. Imre, and H. V. Nguyen, ', '06 May 2019', 17829227, '10.1109/COMST.2017.2786748', 116, 'IEEE', 'https://ieeexplore.ieee.org/document/8242350', '']
idx  122   [-1, 'A single quantum cannot be cloned']   ['A single quantum cannot be cloned', 'W. K. Wootters and W. H. Zurek, ', '', 0, '', 0, '', '', '']
idx  123   [117, 'Entanglement-assisted capacity of a quantum channel and the reverseShannon theorem']   ['Entanglement-assisted capacity of a quantum channel and the reverseShannon theorem', 'C. H. Bennett, P. W. Shor, J. A. Smolin, and A. V. Thapliyal,', '07 November 2002', 7407456, '10.1109/TIT.2002.802612', 372, 'IEEE', 'https://ieeexplore.ieee.org/iel5/18/22215/01035117.pdf', '']
idx  124   [118, 'The private classical capacity and quantum capacity of aquantum channel']   ['The private classical capacity and quantum capacity of aquantum channel', 'I. Devetak, ', '10 

idx  158   [-1, 'Variations on classical andquantum extractors']   ['Variations on classical andquantum extractors', 'M. Berta, O. Fawzi, V. Scholz, and O. Szehr, ', '', 0, '', 0, 'IEEE', '', '']
idx  159   [51, 'The bounded-storage model in thepresence of a quantum adversary']   ['The bounded-storage model in thepresence of a quantum adversary', 'R. T. Konig and B. M. Terhal, ', '', 0, '', 0, 'IEEE', 'https://authors.library.caltech.edu/9660/1/KOEieeetit08.pdf', '']
idx  160   [-1, 'Quantum-proof randomnessextractors via operator space theory']   ['Quantum-proof randomnessextractors via operator space theory', 'M. Berta, O. Fawzi, and V. B. Scholz, ', '', 0, '', 0, 'IEEE', '', '']
idx  161   [-1, 'Extracting randomness viarepeated condensing']   ['Extracting randomness viarepeated condensing', 'R. S. O. Reingold and A. Wigderson, ', '', 0, '', 0, '', '', '']
idx  162   [132, 'Quantum search algorithms,quantum wireless, and a low-complexity maximum likelihood iterativequantum multi-use

idx  200   [144, 'Learn to cache:Machine learning for network edge caching in the big data era']   ['Learn to cache:Machine learning for network edge caching in the big data era', 'Z. Chang, L. Lei, Z. Zhou, S. Mao, and T. Ristaniemi, ', '04 July 2018', 17895127, '10.1109/MWC.2018.1700317', 106, 'IEEE', 'https://ieeexplore.ieee.org/document/8403948', '']
idx  201   [-1, 'A quantumapproximate optimization algorithm']   ['A quantumapproximate optimization algorithm', 'E. Farhi, J. Goldstone, and S. Gutmann. (2014). ', '', 0, '', 0, '', '', '']
idx  202   [145, 'Multi-objective routingoptimization using evolutionary algorithms']   ['Multi-objective routingoptimization using evolutionary algorithms', 'H. Yetgin, K. T. K. Cheung, and L. Hanzo, ', '11 June 2012', 12805181, '10.1109/WCNC.2012.6214324', 28, 'IEEE', 'http://ieeexplore.ieee.org/document/6214324/', '']
idx  203   [-1, 'Multi-task learning as multi-objective optimiza-tion']   ['Multi-task learning as multi-objective optimiza-tion'

idx  230   [158, 'Detec-tion for 5G-NOMA: An online adaptive machine learning approach']   ['Detec-tion for 5G-NOMA: An online adaptive machine learning approach', 'D. A. Awan, R. L. G. Cavalcante, M. Yukawa, and S. Stanczak, ', '31 July 2018', 17971370, '10.1109/ICC.2018.8422449', 21, 'IEEE', 'https://ieeexplore.ieee.org/iel7/8411665/8422068/08422449.pdf', '']
idx  231   [-1, 'The application of machine learningin mmWave-NOMA systems']   ['The application of machine learningin mmWave-NOMA systems', 'J. Cui, Z. Ding, and P. Fan, ', '', 0, '', 0, '', '', '']
idx  232   [159, 'Deep learning for an effec-tive nonorthogonal multiple access scheme']   ['Deep learning for an effec-tive nonorthogonal multiple access scheme', 'G. Gui, H. Huang, Y. Song, and H. Sari, ', '17 September 2018', 18095115, '10.1109/TVT.2018.2848294', 320, 'IEEE', 'https://ieeexplore.ieee.org/document/8387468', '']
idx  233   [-1, 'On the performanceand optimization for MEC networks using uplink NOMA']   ['On the perf

In [7]:
# pd.set_option('display.max_rows', 300)
q1.csv_data.head(300)

,title,author,date,INSPEC,DOI,citations,publisher,url,keywords
0,6G Wireless Communication Systems: Application...,"Mostafa Zaman Chowdhury, Md. Shahjalal, Shakil...",20 July 2020,19955163,10.1109/OJCOMS.2020.3010270,172,IEEE,https://ieeexplore.ieee.org/document/9144301,NaN
1,Terahertz communication for vehicular networks,"S. Mumtaz et al.,",17 July 2017,17026925,10.1109/TVT.2017.2712878,107,IEEE,https://ieeexplore.ieee.org/document/7982949,NaN
2,Quantum machine learning for 6G communica-tion...,"S. J. Nawaz, S. K. Sharma, S. Wyne, M. N. Patw...",15 April 2019,18609246,10.1109/ACCESS.2019.2909490,170,IEEE,https://ieeexplore.ieee.org/document/8681450,
3,Toward 6G networks: Use cases and technologies,"M. Giordani, M. Polese, M. Mezzavilla, S. Rang...",18 March 2020,19469688,10.1109/MCOM.001.1900411,378,IEEE,https://ieeexplore.ieee.org/abstract/document/...,NaN
4,"5G: A tutorial overview of standards, trials, ...","M. Shafi et al.,",02 June 2017,16915203,10.1109/JSAC.2017.2692307,1048,IEEE,https://ieeexplore.ieee.org/document/7894280,NaN
5,Oneintegrated energy efficiency proposal for 5...,"D. Zhang, Z. Zhou, S. Mumtaz, J. Rodriguez, an...",23 May 2017,16585155,10.1109/JIOT.2016.2599852,72,IEEE,https://ieeexplore.ieee.org/document/7542566,NaN
6,5G backhaulchallenges and emerging research di...,"M. Jaber, M. A. Imran, R. Tafazolli, and A. Tu...",23 May 2017,15977764,10.1109/ACCESS.2016.2556011,347,IEEE,http://ieeexplore.ieee.org/document/7456186,NaN
7,What will 5G be,"J. G. Andrews et al.,",22 July 2014,14469237,10.1109/JSAC.2014.2328098,5571,IEEE,https://ieeexplore.ieee.org/document/6824752,NaN
8,6G: The next frontier: From holographic mes-sa...,"E. C. Strinati et al.,",20 August 2019,18920911,10.1109/MVT.2019.2921162,193,IEEE,https://ieeexplore.ieee.org/document/8792135,NaN
9,"A vision of 6G wireless systems:Applications, ...","W. Saad, M. Bennis, and M. Chen,",02 June 2020,19669258,10.1109/MNET.001.1900287,866,IEEE,https://ieeexplore.ieee.org/document/8869705,NaN
